## Whole brain calcium imaging data from C. elegans, Kato et al. 2015

Kato, S., Kaplan, H. S., Schrödel, T., Skora, S., Lindsay, T. H., Yemini, E., et al. (2015). Global Brain Dynamics Embed the Motor Command Sequence of Caenorhabditis elegans. Cell, 163(3), 656–669. http://doi.org/10.1016/j.cell.2015.09.034

In [33]:
!ls
from datetime import datetime
from dateutil.tz import tzlocal

import matplotlib.pyplot as plt
%matplotlib inline 
import pynwb
import math
from hdmf.backends.hdf5.h5_utils import H5DataIO

import h5py
import numpy as np
import scipy.io as sio



all_datarefs = ['WT_Stim']

for data_ref in all_datarefs:
    
    mat_file = '%s.mat'%data_ref
    h5_file = h5py.File(mat_file, 'r')
    mat_contents = h5_file[data_ref]


    print('Contents of MAT file %s: %s'%(mat_file, sorted(mat_contents.keys())))
    
    '''
    From readme:
    
    traces_raw= neural activity traces uncorrected
    traces = neural activity traces corrected for bleaching
    tracesDif = derivative of traces
    IDs = identified neuron IDs
    timeVectorSeconds = time vector in seconds
    fps = frames per second
    dataset = name of dataset
    stimulus
        -identity = what was changed e.g. O2 (oxygen)
        -type = stimulus type e.g. binary steps
        -switchtimes =  time in seconds when stimulus changed from initial state to the other state
        -initialstate = the state that the stimulus starts with, refers to "conc"
        -conc = the concentrations of the stimulus
        -concunits - units of the "conc"

    States = vector of different state types (8 states for WT_NoStim, 4 states for WT_Stim and AVA_HisCl

    8 states for WT_NoStim:
        ‘FWD’ forward crawling; ‘SLOW’ forward slowing; ‘DT’ dorsal post reversal turn
        ‘VT’ ventral post reversal turn; ‘REV1’ reverse crawling;
        ‘REV2’ reverse crawling; ‘REVSUS’ sustained reverse crawling; ‘NOSTATE’ - ambiguous

    4 states for WT_Stim and AVA_HisCl:
        ‘FWD’ forward crawling; ‘REV’ reverse crawling
        ‘REVSUS’ sustained reverse crawling; ‘TURN’ post reversal turn
    '''
    
    for dataset_index in [0,1]:
        
        start_time = datetime.now(tz=tzlocal())
        create_date = datetime.now(tz=tzlocal())

        experimenter = '??? (Zimmer lab)'

        main_ref = 'Kato et al. 2015 dataset taken from file %s.mat'%data_ref
        nwbfile = pynwb.NWBFile(main_ref, 
                      main_ref, 
                      start_time,
                      file_create_date=create_date,
                      notes='NWB file created with pynwb v%s'%pynwb.__version__,
                      experimenter=experimenter,
                      experiment_description='ED...',
                      institution='IN...')
        
        print('\n========  loading fps %i'%dataset_index)
        fps_raw_h5ref = np.array(mat_contents['fps'])
        fps_raw = h5_file[fps_raw_h5ref[dataset_index][0]]
        #print(fps_raw)
        print(fps_raw[0][0])
        

        print('\n========  loading dataset %i'%dataset_index)

        raw_h5ref = np.array(mat_contents['dataset'])
        raw = h5_file[raw_h5ref[dataset_index][0]]
        #print(raw)
        st = [r[0] for r in raw]
        s2 = ''.join(map(chr,st))
        print(s2)

        print('\n========  loading timeVectorSeconds %i'%dataset_index)

        raw_h5ref = np.array(mat_contents['timeVectorSeconds'])
        raw = h5_file[raw_h5ref[dataset_index][0]]
        print('Data points %s: %s'%(raw.shape, raw[0]))
        timestamps = raw[0]

        print('\n========  loading States %i'%dataset_index)

        raw_h5ref = np.array(mat_contents['States'])
        raw = h5_file[raw_h5ref[dataset_index][0]]
        states = np.array([s[0] for s in raw])
        print('Data points %s: %s'%(len(states), states))
        
        data = states

        comments='Extracted from MAT file: %s.mat'%data_ref
        wrapped_data = H5DataIO(data=data, compression=True) 

        ts_acq = pynwb.TimeSeries('States', wrapped_data, 'none', timestamps=timestamps,comments=comments,
                                 description='vector of different state types')
        nwbfile.add_acquisition(ts_acq)


        nwb_file_name = 'KatoEtAl2018.%s.%i.nwb'%(data_ref, dataset_index)
        io = pynwb.NWBHDF5IO(nwb_file_name, mode='w')
        io.write(nwbfile)
        io.close()
        print("Written NWB file to %s"%nwb_file_name)
    
    '''
    key_set = sorted(mat_contents.keys())
    traces_raw_h5ref = mat_contents['traces_raw']
    traces_raw = h5_file[traces_raw_h5ref[0][0]]
    print(traces_raw)
    print(traces_raw.value)
    print(traces_raw.value[0])
    

    print('IDs of cells: ')
    IDs_h5ref = mat_contents['IDs']
    IDs = h5_file[IDs_h5ref[0][0]]
    print(IDs)
    #print(IDs.value)
    print(IDs[0])
    print(h5_file[IDs[0][0]].value)

    
    plt.figure()
    id = '???'
    for i in range(15,25,1):
        id = h5_file[IDs[i][0]].value[0]
        print('ID: %s (%s)'%(id, type(id))) 
        if not type(id)==np.uint64:
            id = [int(a) for a in h5_file[id[0]].value]
            id = ''.join(chr(i) for i in id)
        else:
            id = '???'
        print('Plotting cell %i: %s'%(i,id))
        plt.plot(timeVectorSeconds.value[0],traces_raw.value[i], lw=.5, label='%i: %s'%(i,id))
        plt.legend()

        
    plt.show()    '''


AVA_HisCl.mat       WT_NoStim.mat       WT_Stim.mat
TestData.ipynb      WT_NoStim_pre73.mat readme_Kato2015.txt
Contents of MAT file WT_Stim.mat: ['IDs', 'States', 'dataset', 'fps', 'stimulus', 'timeVectorSeconds', 'traces', 'tracesDif', 'traces_raw']

========  loading fps 0
3.0527777777777776

========  loading dataset 0
TS20140624b_lite-1_punc-31_NLS3_4eggs_56um_1mMTet_basalplusstim_720s

========  loading timeVectorSeconds 0
Data points (1, 2198): [0.00000000e+00 3.27570519e-01 6.55141037e-01 ... 7.19017288e+02
 7.19344859e+02 7.19672429e+02]

========  loading States 0
Data points 2198: [4. 4. 4. ... 1. 1. 1.]


/Users/padraig/anaconda/envs/py36/lib/python3.6/site-packages/hdmf/build/objectmapper.py:137: UserWarning: Value with data type float64 is being converted to data type float64 as specified.
  % (g.name, s.name))


Written NWB file to KatoEtAl2018.WT_Stim.0.nwb

========  loading fps 1
2.801388888888889

========  loading dataset 1
TS20140630h_lite-1_punc-31_NLS3_2eggs_56um_1mMTet_basalplusstim_720s

========  loading timeVectorSeconds 1
Data points (1, 2017): [0.00000000e+00 3.56965791e-01 7.13931582e-01 ... 7.18929103e+02
 7.19286068e+02 7.19643034e+02]

========  loading States 1
Data points 2017: [1. 1. 1. ... 1. 1. 1.]
Written NWB file to KatoEtAl2018.WT_Stim.1.nwb


In [ ]:
from datetime import datetime

from dateutil.tz import tzlocal
import pynwb
import math
from hdmf.backends.hdf5.h5_utils import H5DataIO

print('Using pynwb v%s'%pynwb.__version__)
'''
for data_ref in all_datarefs:

    mat_file = '%s.mat'%data_ref
    mat_contents = sio.loadmat(mat_file)

    start_time = datetime(2019, 1, 1, 11, tzinfo=tzlocal())
    create_date = datetime.now(tz=tzlocal())

    experimenter = 'Angelica da Silva Lantyer' if 'AL' in data_ref else \
                   ('Niccolò Calcini' if 'NC' in data_ref else \
                    ('Melanie Emmelkamp' if 'ME' in data_ref else '???'))

    main_ref = 'Lantyer et al. dataset taken from file %s.mat'%data_ref
    nwbfile = pynwb.NWBFile(main_ref, 
                      main_ref, 
                      start_time,
                      file_create_date=create_date,
                      notes='NWB file created with pynwb v%s'%pynwb.__version__,
                      experimenter=experimenter,
                      experiment_description='A databank for intracellular electrophysiological mapping of the adult somatosensory cortex',
                      institution='Donders Institute for Brain, Cognition and Behaviour, Radboud University')


    for t in mat_contents.keys():
        if 'Trace_' in t:
            print('Adding trace: %s'%t)
            trace = mat_contents[t].T

            #plt.plot(trace[0],trace[1], lw=.5, label='%s'%t)

            timestamps = trace[0]
            data = trace[1]

            comments='Extracted from MAT file: %s.mat'%data_ref
            
            wrapped_data = H5DataIO(data=data, compression=True) 
            #wrapped_data = data 

            if t.endswith('_1'):
                ts_stim = pynwb.TimeSeries('%s'%t, wrapped_data, 'A', timestamps=timestamps,comments=comments,
                                          description='Membrane potential')
                nwbfile.add_stimulus(ts_stim)
            if t.endswith('_2'):
                ts_acq = pynwb.TimeSeries('%s'%t, wrapped_data, 'V', timestamps=timestamps,comments=comments,
                                         description='Applied current')
                nwbfile.add_acquisition(ts_acq)


    nwb_file_name = 'LantyerEtAl2018.%s.nwb'%(data_ref)
    io = pynwb.NWBHDF5IO(nwb_file_name, mode='w')
    io.write(nwbfile)
    io.close()
    print("Written NWB file to %s"%nwb_file_name)'''